In [1]:
# pip install pandas xlrd
import pandas as pd # We import pandas to work with dataframes.
import numpy as np # We import numpy to work with matrix.
import xlrd # We import xlrd because pandas need it due to our original data-table is in a old version
import re # We import regex library for searching patterns. 

# We load the data-table excel from the url.
url = 'https://www.sharkattackfile.net/spreadsheets/GSAF5.xls'
data_df = pd.read_excel(url)

In [2]:
# We attach to our main a python document which is keeping the main function we need to use. 
import sys
sys.path.append("SHARKATTACKSPROYECT/")
from functions import *


In [6]:
# We are taking a snapshot about our data panel in order to see to what we are facing.

data_df.shape

(6969, 23)

In [ ]:
# It seems before step is enough to take a complete snapshot of our data. Lets go with another method.

pd.options.display.max_columns # We get access to configure display options from Pandas and configure max columns to show.

In [ ]:
# Once we've seen the columns distribution, we gonna face the date types of each column to know how start to work with each one.
# We need to know how many Na/Non Values we have in data.
data_df.info()

# YEAR AND DATE COLUMNS

In [ ]:
# In column YEAR, lets see how many unique values it contains: 
data_df.Year.unique()

In [3]:
# There are some years with no values, so we will try if we can confirm the year though column Date. 
data_df[['Date','Year']][data_df.Year == 0] #81 years we can rescue. 

,Date,Year
6815,Ca. 214 B.C.,0.0
6816,Ca. 336.B.C..,0.0
6817,Ca. 493 B.C.,0.0
6818,Ca. 725 B.C.,0.0
6819,Ca. 1010 BC,0.0
...,...,...
6939,Before 1903,0.0
6940,Before 1903,0.0
6941,1900-1905,0.0
6942,1883-1889,0.0


In [ ]:
# Lets try to aply some functions in order to rescue these years:
'''Buscamos mediante un regex, para que nos busque los años numéricos'''

data_df.Date = data_df.Date.replace(regex=r'(?i)Reported\s{1,9}',value='')
list(data_df.Date.unique())

In [5]:
# Aunque no salen en los uniques hay muchos Dates que salen como rangos, o datos de antes de Cristo.
# Vamos a denominarlos momentáneamente salvables para ver cuantos hay exactamente:
salvables = data_df.loc[(data_df["Year"] == 0) & (data_df["Date"] != np.nan)]
salvables.shape

(129, 23)

In [6]:
# Llamamos a las funciones definidas en el cleanin_functions.py, mas concretamente a rescatar 
# fechas que aplica 3 funciones secuencialmente donde coge por orden los que contienen BC, 
# los que son fechas sueltas (tipo Before YYYY) y luego los intervalos de los cuales saca la media, 
# para rellenar los datos de la columna Year para esos valores de Dates 
from functions import *


In [7]:
data_df.Year = data_df.Date.apply(rescatar_fechas)

In [ ]:
data_df.Year=rescatar_fechas(data_df.Date)

In [9]:
data_df_year = data_df.Year

In [ ]:
# Eliminamos todas las filas con valores NA

data_df_year.dropna(axis=0, inplace= True, how='all')

In [20]:
data_df_year.to_csv('datadavid1.csv', columns=['Year'], index=False)

In [18]:
data_df_year

0       2024
1       2024
2       2024
3       2024
4       2024
        ... 
6939    1903
6940    1903
6941    1900
6942    1883
6943    1845
Name: Year, Length: 6925, dtype: object

In [ ]:
data_df_year.unique()

In [12]:
data_df_year

0       2024
1       2024
2       2024
3       2024
4       2024
        ... 
6939    1903
6940    1903
6941    1900
6942    1883
6943    1845
Name: Year, Length: 6925, dtype: object

In [ ]:
# Lets see to get a general idea about the problem with Na in order to clean approaching. We get percentages.

Na_amount = round(data_df.isna().sum()*100/len(data_df), 2)
Na_amount.sort_values(ascending = False)

In [11]:
# We see there are some columns with a too high percentage of Nan Values.
# A logical idea we can conclude, it is those columns with a high percentage of nan values are useless due to missinformation given.
# So We decide to drop columns named "Unnamed 21" and "Unnamed 22".





In [ ]:


df.dropna(axis=0, inplace= True, thresh=int(24*0.8))

In [ ]:
# Análisis de los datos de fecha

#Columna Date

#Vemos los valores únicos
data_df.Year.unique()

In [ ]:
# Tenemos valores muy raros, como año 0, 5, 77. Por cuestiones históricas, decidimos dropear todo lo anterior al año 1800.

# Como tenemos otra columna (date) que también nos da años, intentamos si podemos rescatar datos: 

data_df[['Date','Year']][data_df.Year == 0] #81 fechas que podemos rescatar a través de date

In [ ]:
data_df.Date = data_df.Date.replace(regex=r'(?i)Reported\s{1,9}',value='')
list(data_df.Date.unique())

In [ ]:
data_df.Date.str.replace("-", " ") #1
data_df.Date.str.replace("_", " ")#2
data_df.Date.str.strip() #3
